In [10]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats as st

In [ ]:
#mds_data= pd.read_csv("MDS_original_data_NA_clean.csv")
mds_data= pd.read_csv("MDS_added_new_vars_MDS_data.csv")
mds_data.head()

,patinit,patnum,picode,assessdate,ybirth,age,sex,educat,yeduc,civilst,...,nmstot,mdsupdrs_I,mdsupdrs_II,mdsupdrs_III,mdsupdrs_IV,cisipd,age_c,duratpd_c,epworth_mdsnms,depression_sum
0,MSS,1,3,10/26/2016,1945,71,0,4,20,2,...,125.0,20.0,27.0,44.0,5.0,14.0,2,3,9,7.0
1,TF,10,3,11/28/2016,1948,68,0,4,18,2,...,54.0,5.0,14.0,28.0,2.0,6.0,2,2,0,0.0
2,SP,100,3,12/11/2017,1936,81,0,4,20,2,...,39.0,11.0,14.0,23.0,1.0,9.0,3,3,4,0.0
3,MK,101,3,1/12/2018,1956,61,0,4,17,2,...,105.0,26.0,25.0,29.0,6.0,9.0,2,3,9,4.0
4,LK,102,3,1/16/2018,1954,63,1,4,16,2,...,109.0,23.0,18.0,31.0,5.0,11.0,2,3,6,4.0


In [12]:
print(list(mds_data.columns).index('sex'))

6


In [13]:
mod_val_list= []

for row_id in range(mds_data.shape[0]):
    if mds_data.iloc[row_id, 6] == 0:
        mds_data.iloc[row_id, 6] = 2

In [14]:
from collections import Counter

In [15]:
print(Counter(mds_data['sex']))

Counter({2: 250, 1: 152})


In [ ]:
lrrk2_pos_data_miss30= pd.read_csv("lrrk2_pos_data_miss30.csv")
lrrk2_pos_data_miss30.head()

,datasrc,lrrkid,event_id,pdenrl,ageonset,demopd_ageassess,demopd_ageassess_gt89,MCAVFNUM,P1IMPRF,P1THTDSF,...,alcohol_ageassess,antiinfl_ageassess,upsitbk1,upsitbk2,upsitbk3,updrs1_ageassess,updrs1_ageassess_gt89,upsitbk4,biodadpd,CIGLF100
0,1,10001,V01,1,55,61,0,Z,Z,Z,...,Z,Z,NaN,NaN,NaN,61.0,0.0,NaN,0,1.0
1,2,10002,V01,1,52,70,0,Z,Z,Z,...,NaN,NaN,Z,Z,Z,NaN,NaN,Z,NaN,0.0
2,4,10004,V01,1,55,74,0,Z,Z,Z,...,74,74,4,5,6,74.0,0.0,3,0,1.0
3,6,10006,V01,1,52,75,0,Z,Z,Z,...,75,75,1,4,0,75.0,0.0,3,0,0.0
4,9,10009,V01,1,46,54,0,Z,Z,Z,...,54,54,Z,Z,Z,54.0,0.0,Z,Z,NaN


In [ ]:
print(lrrk2_pos_data_miss30.shape)

(1470, 443) (1470, 505)


In [21]:
if 'depression_sum' in mds_data.columns:
    print("Yes")
else:
    print("No")

Yes


# Data preprocessing to perform feature mapping

### Modification required for mapping
sex in MDS: male = 0, female = 1
gender in LRRK2: 1=Female, 2=Male

In [23]:
features_to_remove_anyway= ['moca_ageassess','gds_ageassess','diagchk_ageassess','scopaaut_ageassess','modseadl_ageassess','modhoya_ageassess','updrs1_ageassess','birthyr','updrs3_ageassess','updrs2_ageassess', 'caffein_ageassess','smoktob_ageassess','updrs4_ageassess','imgaddl_ageassess', 'lrrkid', 'datasrc', 'event_id', 'birthyr', 'racetxt']


In [24]:
mds_updrs_features = ['NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT', 'NP1DDS', 'nupdrs1_ageassess', 
                      'nupdrs1_ageassess_gt89', 'NP1SLPN', 'NP1SLPD', 'NP1PAIN', 'NP1URN', 'NP1CNST', 'NP1LTHD', 'NP1FATG',
                      'nupdrs1p_ageassess', 'nupdrs1p_ageassess_gt89', 'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES',
                      'NP2HYGN', 'NP2HWRT', 'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE', 'NP2WALK', 'NP2FREZ', 'nupdrs2p_ageassess',
                      'nupdrs2p_ageassess_gt89', 'PDCLNSTA', 'LDOPARX', 'MINSNCLD', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN',
                      'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL',
                      'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT',
                      'NP3PSTBL', 'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU', 
                      'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES', 'DYSKIRAT', 'NHY', 'nupdrs3_ageassess',
                      'nupdrs3_ageassess_gt89', 'NP4WDYSK', 'NP4DYSKI', 'NP4OFF', 'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN', 'nupdrs4_ageassess',
                      'nupdrs4_ageassess_gt89']




In [25]:
moca_features = ['subgrpid', 'ageonset', 'MODHY', 'GDS15SCORE', 'pdenrl', 'MCAALTTM', 'MCACUBE', 'MCACLCKC', 'MCACLCKN', 'MCACLCKH', 'MCALION', 'MCARHINO', 'MCACAMEL', 
                 'MCAFDS', 'MCABDS', 'MCASER7', 'MCASNTNC', 'MCAVFNUM', 'MCAVF', 'MCAABSTR', 'MCAREC1',
                 'MCAREC2', 'MCAREC3', 'MCAREC4', 'MCAREC5', 'MCADATE', 'MCAMONTH', 'MCAYR', 'MCADAY', 'MCAPLACE',
                 'MCACITY', 'MCATOT', 'moca_ageassess', 'moca_ageassess_gt89']


In [26]:
epworth_sleepiness_features = ['subgrpid', 'ageonset', 'epworth_ptcgboth', 'ESS1', 'ESS2', 'ESS3', 'ESS4', 'ESS5', 'ESS6', 'ESS7',
                               'ESS8', 'epworth_ageassess', 'epworth_ageassess_gt89']

In [27]:
mapped_features_in_lrrk2= {'pd_duration': 'durat_pd', 'ageonset': 'pdonset', 'demopd_ageassess': 'age', 'gender': 'sex', 'MCATOT': 'moca', 'NHY': 'hy', 'totled': 'ldopadose__LED_', 'epworth_sum': 'epworth_mdsnms', 'GDS15SCORE': 'depression_sum'}

mds_updrs_mapping= {'NP1COG': 'mdsupdrs1_1', 'NP1HALL': 'mdsupdrs1_2', 'NP1DPRS': 'mdsupdrs1_3', 'NP1ANXS': 'mdsupdrs1_4', 'NP1APAT': 'mdsupdrs1_5', 'NP1DDS': 'mdsupdrs1_6', 'NP1SLPN': 'mdsupdrs1_7', 'NP1SLPD': 'mdsupdrs1_8', 'NP1PAIN': 'mdsupdrs1_9', 'NP1URN': 'mdsupdrs1_10', 'NP1CNST': 'mdsupdrs1_11', 'NP1LTHD': 'mdsupdrs1_12', 'NP1FATG': 'mdsupdrs1_13', 'NP2SPCH': 'mdsupdrs2_1', 'NP2SALV': 'mdsupdrs2_2', 'NP2SWAL': 'mdsupdrs2_3', 'NP2EAT': 'mdsupdrs2_4', 'NP2DRES': 'mdsupdrs2_5', 'NP2HYGN': 'mdsupdrs2_6', 'NP2HWRT': 'mdsupdrs2_7', 'NP2HOBB': 'mdsupdrs2_8', 'NP2TURN': 'mdsupdrs2_9', 'NP2TRMR': 'mdsupdrs2_10', 'NP2RISE': 'mdsupdrs2_11', 'NP2WALK': 'mdsupdrs2_12', 'NP2FREZ': 'mdsupdrs2_13', 'PDMEDYN': 'mdsupdrs3a', 'PDCLNSTA': 'mdsupdrs3b', 'LDOPARX': 'mdsupdrs3c', 'MINSNCLD': 'mdsupdrs3c1', 'NP3SPCH': 'mdsupdrs3_1', 'NP3FACXP': 'mdsupdrs3_2', 'NP3RIGN': 'mdsupdrs3_3a', 'NP3RIGRU': 'mdsupdrs3_3b', 'NP3RIGLU': 'mdsupdrs3_3c', 'NP3RIGRL': 'mdsupdrs3_3d', 'NP3RIGLL': 'mdsupdrs3_3e', 'NP3FTAPR': 'mdsupdrs3_4a', 'NP3FTAPL': 'mdsupdrs3_4b', 'NP3HMOVR': 'mdsupdrs3_5a', 'NP3HMOVL': 'mdsupdrs3_5b', 'NP3PRSPR': 'mdsupdrs3_6a', 'NP3PRSPL': 'mdsupdrs3_6b', 'NP3TTAPR': 'mdsupdrs3_7a', 'NP3TTAPL': 'mdsupdrs3_7b', 'NP3LGAGR': 'mdsupdrs3_8a', 'NP3LGAGL': 'mdsupdrs3_8b', 'NP3RISNG': 'mdsupdrs3_9', 'NP3GAIT': 'mdsupdrs3_10', 'NP3FRZGT': 'mdsupdrs3_11', 'NP3PSTBL': 'mdsupdrs3_12', 'NP3POSTR': 'mdsupdrs3_13', 'NP3BRADY': 'mdsupdrs3_14', 'NP3PTRMR': 'mdsupdrs3_15a', 'NP3PTRML': 'mdsupdrs3_15b', 'NP3KTRMR': 'mdsupdrs3_16a', 'NP3KTRML': 'mdsupdrs3_16b', 'NP3RTARU': 'mdsupdrs3_17a', 'NP3RTALU': 'mdsupdrs3_17b', 'NP3RTARL': 'mdsupdrs3_17c', 'NP3RTALL': 'mdsupdrs3_17d', 'NP3RTALJ': 'mdsupdrs3_17e', 'NP3RTCON': 'mdsupdrs3_18', 'DYSKPRES': 'mdsupdrs_dysk', 'DYSKIRAT': 'mdsupdrs_int', 'NHY': 'mdsupdrs_hy', 'NP4WDYSK': 'mdsupdrs4_1', 'NP4DYSKI': 'mdsupdrs4_2', 'NP4OFF': 'mdsupdrs4_3', 'NP4FLCTI': 'mdsupdrs4_4', 'NP4FLCTX': 'mdsupdrs4_5', 'NP4DYSTN': 'mdsupdrs4_6'}


In [28]:
total_mapped_features= list(mapped_features_in_lrrk2.keys()) + list(mds_updrs_mapping.keys())
print(total_mapped_features)

['pd_duration', 'ageonset', 'demopd_ageassess', 'gender', 'MCATOT', 'NHY', 'totled', 'epworth_sum', 'GDS15SCORE', 'NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT', 'NP1DDS', 'NP1SLPN', 'NP1SLPD', 'NP1PAIN', 'NP1URN', 'NP1CNST', 'NP1LTHD', 'NP1FATG', 'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES', 'NP2HYGN', 'NP2HWRT', 'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE', 'NP2WALK', 'NP2FREZ', 'PDMEDYN', 'PDCLNSTA', 'LDOPARX', 'MINSNCLD', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES', 'DYSKIRAT', 'NHY', 'NP4WDYSK', 'NP4DYSKI', 'NP4OFF', 'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN']


In [29]:
print(len(total_mapped_features))

81


In [30]:
# overlap with LRRK2 features with less than 30% missing data
overlap_miss30_features= set(lrrk2_pos_data_miss30.columns).intersection(set(total_mapped_features))


In [ ]:
print(len(overlap_miss30_features))

In [33]:
# We plan to go with the features with less than 30% missing data
# First let's see the mapped features that we will be missing

difference_miss30_features= set(total_mapped_features).difference(set(lrrk2_pos_data_miss30.columns))
print(difference_miss30_features)

{'MCATOT', 'epworth_sum', 'PDMEDYN'}


In [35]:
lrrk2_final_ready= lrrk2_pos_data_miss30[list(overlap_miss30_features)]

In [36]:
lrrk2_final_ready.head()

,NP3TTAPL,NP2SWAL,NP1SLPN,NP1COG,NP3RISNG,NP3FRZGT,NP2TRMR,NP2EAT,NP3FTAPR,NP1SLPD,...,NP3RIGRL,gender,NP3GAIT,NP2SPCH,NP4OFF,NP3TTAPR,DYSKPRES,NP3LGAGR,NP3KTRMR,NP3POSTR
0,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,Z,2,Z,Z,Z,Z,Z,Z,Z,Z
1,4,4,0,0,4,4,4,3,4,2,...,4,2,4,4,4,3,0,3,0,3
2,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,Z,1,Z,Z,Z,Z,Z,Z,Z,Z
3,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,Z,1,Z,Z,Z,Z,Z,Z,Z,Z
4,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,Z,2,Z,Z,Z,Z,Z,Z,Z,Z


In [ ]:
lrrk2_final_ready.to_csv("lrrk2_data_mds_mapped_features_only.csv", index=False)

In [ ]:
#lrrk2_final_ready= pd.read_csv("lrrk2_data_mds_mapped_features_only.csv")

In [38]:
mds_updrs_keys_present= set(list(mds_updrs_mapping.keys())).intersection(set(lrrk2_pos_data_miss30.columns))

In [39]:
lrrk2_final_ready_mds_updrs= lrrk2_pos_data_miss30[list(mds_updrs_keys_present)]

In [40]:
lrrk2_final_ready_mds_updrs[lrrk2_final_ready_mds_updrs == 'Z'] = np.nan

C:\Users\roysoumya\AppData\Local\Temp\ipykernel_11136\2199017677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lrrk2_final_ready_mds_updrs[lrrk2_final_ready_mds_updrs == 'Z'] = np.nan
C:\Users\roysoumya\AppData\Local\Temp\ipykernel_11136\2199017677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lrrk2_final_ready_mds_updrs[lrrk2_final_ready_mds_updrs == 'Z'] = np.nan


In [41]:
lrrk2_final_ready_mds_updrs.head()

,NP3TTAPL,NP2SWAL,NP3RIGRU,PDCLNSTA,NP1SLPN,NP1DPRS,NP1COG,NP3FACXP,NP3RTARL,NHY,...,NP3LGAGR,DYSKPRES,NP3FTAPL,NP3KTRML,NP3KTRMR,DYSKIRAT,NP4WDYSK,NP1LTHD,MINSNCLD,NP3POSTR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,4,3,NaN,0,3,0,3,0,5,...,3,0,4,0,0,0,0,2,720,3
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# dropping rows with all columns as NaN
lrrk2_final_ready_mds_updrs1= lrrk2_final_ready_mds_updrs.dropna(how='all')
print(lrrk2_final_ready_mds_updrs.shape, lrrk2_final_ready_mds_updrs1.shape)

(1470, 71) (397, 71)


### Only consider patients with MDS-UPDRS information --- remove patient with all NaN values under the MDS-UPDRS heading

In [44]:
lrrk2_final_ready1= lrrk2_final_ready
lrrk2_final_ready1[lrrk2_final_ready1 == 'Z'] = np.nan

mdsupdrs_row_all_nan= []

for row_id in range(lrrk2_final_ready1.shape[0]):
    mdsupdrs_row_all_nan.append(lrrk2_final_ready1[list(mds_updrs_keys_present)].isnull().values[row_id,:].all())

In [45]:
from collections import Counter
print(Counter(mdsupdrs_row_all_nan))

Counter({True: 1073, False: 397})


In [46]:
lrrk2_final_ready1= lrrk2_final_ready1.assign(mdsupdrs_present_check= mdsupdrs_row_all_nan)

In [47]:
# Keep patients with postive mdsupdrs values
lrrk2_final_ready2= lrrk2_final_ready1.loc[lrrk2_final_ready1['mdsupdrs_present_check'] == False]
print(lrrk2_final_ready2.shape)

(397, 78)


In [48]:
lrrk2_final_ready2.head()

,NP3KTRMR,NP1ANXS,demopd_ageassess,GDS15SCORE,LDOPARX,NP1SLPN,NP3POSTR,NP2DRES,NP3RIGRL,NP1HALL,...,NP3PSTBL,NP3FTAPL,NP4DYSKI,NP3RTALJ,pd_duration,NP3FRZGT,NP2TRMR,NP3RIGRU,DYSKIRAT,mdsupdrs_present_check
1,0,3,70,10.0,1,0,3,4,4,0,...,4,4,0,0,18,4,4,3,0,False
6,0,0,64,0.0,1,0,1,3,1,0,...,0,2,0,0,4,0,1,1,0,False
19,0,0,61,8.0,1,2,0,0,2,0,...,0,3,0,0,2,0,2,2,0,False
23,0,2,28,12.0,1,2,1,0,0,0,...,0,1,0,0,8,0,2,0,NaN,False
25,0,0,55,0.0,1,0,1,1,1,0,...,0,1,0,0,2,0,2,1,0,False


In [49]:
lrrk2_final_ready2.columns

Index(['NP3KTRMR', 'NP1ANXS', 'demopd_ageassess', 'GDS15SCORE', 'LDOPARX',
       'NP1SLPN', 'NP3POSTR', 'NP2DRES', 'NP3RIGRL', 'NP1HALL', 'NP3FTAPR',
       'NP3RIGLU', 'NP1DDS', 'NP3RIGN', 'NP1PAIN', 'NP1APAT', 'NP3RTALL',
       'NP4FLCTX', 'PDCLNSTA', 'NP3RISNG', 'gender', 'NP3BRADY', 'NP3TTAPR',
       'NP4FLCTI', 'NP3PTRMR', 'NP3SPCH', 'NP3PRSPR', 'NP3TTAPL', 'NP2SWAL',
       'NP3RTARL', 'NP3PTRML', 'DYSKPRES', 'NP1SLPD', 'NP4OFF', 'NHY',
       'NP1FATG', 'NP3RTARU', 'NP1LTHD', 'NP3HMOVR', 'NP3PRSPL', 'NP3GAIT',
       'NP3LGAGR', 'NP2HWRT', 'NP1URN', 'MINSNCLD', 'NP1DPRS', 'NP1COG',
       'NP2SPCH', 'NP4WDYSK', 'NP3LGAGL', 'NP2TURN', 'totled', 'NP1CNST',
       'NP3KTRML', 'NP3RTALU', 'NP3RIGLL', 'NP2RISE', 'NP3RTCON', 'NP2HOBB',
       'NP2SALV', 'NP3FACXP', 'NP2EAT', 'ageonset', 'NP4DYSTN', 'NP2WALK',
       'NP2FREZ', 'NP2HYGN', 'NP3HMOVL', 'NP3PSTBL', 'NP3FTAPL', 'NP4DYSKI',
       'NP3RTALJ', 'pd_duration', 'NP3FRZGT', 'NP2TRMR', 'NP3RIGRU',
       'DYSKIRAT', 'mdsupdrs_

In [50]:
lrrk2_final_ready2= lrrk2_final_ready2.drop(['mdsupdrs_present_check', 'totled'], axis=1)

## Prepare ageonset variable as target label

### after 50 years: late onset

### before 50 years and above 20 years: early onset

### less than or equal to 20 years: juvenile onset

### Remove patients with age onset equal to 50 years


In [ ]:
#lrrk2_final_ready2= pd.read_csv('lrrk2_data_analysis.csv')

In [52]:
lrrk2_final_ready3 = lrrk2_final_ready2.drop(lrrk2_final_ready2[lrrk2_final_ready2.ageonset == 50].index)
print(lrrk2_final_ready3.shape)

(385, 76)


In [53]:
ageonset_col_id= lrrk2_final_ready3.columns.get_loc('ageonset')
print(ageonset_col_id)

61


In [54]:
ageonset_label_list= []
for row_id in range(lrrk2_final_ready3.shape[0]):
    if lrrk2_final_ready3.iloc[row_id, ageonset_col_id] > 50:
        ageonset_label_list.append('late-onset')
    elif lrrk2_final_ready3.iloc[row_id, ageonset_col_id] <= 20:
        ageonset_label_list.append('juvenile-onset')
    elif lrrk2_final_ready3.iloc[row_id, ageonset_col_id] < 50:
        ageonset_label_list.append('early-onset')

In [55]:
print(Counter(ageonset_label_list))

Counter({'late-onset': 254, 'early-onset': 127, 'juvenile-onset': 4})


In [56]:
lrrk2_final_ready3= lrrk2_final_ready3.assign(ageonset_label= ageonset_label_list)
lrrk2_final_ready3= lrrk2_final_ready3.drop(['ageonset'], axis=1)


## Simple missing data imputation using median value of the respective column

### But first we must divide it into train and test set

In [57]:
lrrk2_final_ready4 = lrrk2_final_ready3.drop(lrrk2_final_ready3[lrrk2_final_ready3.ageonset_label == 'juvenile-onset'].index)
print(lrrk2_final_ready4.shape)

(381, 76)


In [58]:
X= lrrk2_final_ready4.drop(['ageonset_label'], axis=1)
y= lrrk2_final_ready4['ageonset_label']

In [59]:
print(X.dtypes)

NP3KTRMR              int64
NP1ANXS               int64
demopd_ageassess      int64
GDS15SCORE          float64
LDOPARX               int64
                     ...   
pd_duration           int64
NP3FRZGT              int64
NP2TRMR               int64
NP3RIGRU            float64
DYSKIRAT            float64
Length: 75, dtype: object


In [60]:
# print the column names of X_train which contain at least one null value or NA value or empty value
print(X.columns[X.isnull().any()])

# compute the percentage of NA values in each column of X_train and X_test
column_vs_miss_percent= X.isnull().sum()/X.shape[0]
#print(X_test.isnull().sum()/X_test.shape[0])

Index(['GDS15SCORE', 'NP1SLPN', 'NP3RIGRL', 'NP3FTAPR', 'NP3RIGLU', 'NP3RIGN',
       'NP1PAIN', 'NP3RTALL', 'PDCLNSTA', 'NP3TTAPR', 'NP3PRSPR', 'NP3TTAPL',
       'NP3RTARL', 'NP1SLPD', 'NP1FATG', 'NP3RTARU', 'NP1LTHD', 'NP3HMOVR',
       'NP3PRSPL', 'NP1URN', 'MINSNCLD', 'NP1CNST', 'NP3RTALU', 'NP3RIGLL',
       'NP3HMOVL', 'NP3RTALJ', 'NP3RIGRU', 'DYSKIRAT'],
      dtype='object')


In [61]:
print(X.shape)

(381, 75)


In [62]:
type(column_vs_miss_percent)

pandas.core.series.Series

In [63]:
column_vs_miss_percent_df= pd.DataFrame(column_vs_miss_percent)

In [64]:
column_vs_miss_percent_df.shape

(75, 1)

In [ ]:
column_vs_miss_percent_df.to_csv('column_vs_miss_percent_df_1.csv')

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [67]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(304, 75) (77, 75) (304,) (77,)


In [ ]:
#export X_train and X_test as csv files
X_train.to_csv('X_train_1.csv', index=False)
X_test.to_csv('X_test_1.csv', index=False)

In [69]:
# print the column names of X_train which contain at least one null value or NA value or empty value
print(X_train.columns[X_train.isnull().any()])

# compute the percentage of NA values in each column of X_train and X_test
print(X_train.isnull().sum()/X_train.shape[0])
print(X_test.isnull().sum()/X_test.shape[0])

Index(['GDS15SCORE', 'NP1SLPN', 'NP3RIGRL', 'NP3FTAPR', 'NP3RIGLU', 'NP3RIGN',
       'NP1PAIN', 'NP3RTALL', 'PDCLNSTA', 'NP3PRSPR', 'NP3RTARL', 'NP1SLPD',
       'NP1FATG', 'NP3RTARU', 'NP1LTHD', 'NP3HMOVR', 'NP3PRSPL', 'NP1URN',
       'MINSNCLD', 'NP1CNST', 'NP3RTALU', 'NP3RIGLL', 'NP3HMOVL', 'NP3RIGRU',
       'DYSKIRAT'],
      dtype='object')
NP3KTRMR            0.000000
NP1ANXS             0.000000
demopd_ageassess    0.000000
GDS15SCORE          0.039474
LDOPARX             0.000000
                      ...   
pd_duration         0.000000
NP3FRZGT            0.000000
NP2TRMR             0.000000
NP3RIGRU            0.003289
DYSKIRAT            0.078947
Length: 75, dtype: float64
NP3KTRMR            0.000000
NP1ANXS             0.000000
demopd_ageassess    0.000000
GDS15SCORE          0.129870
LDOPARX             0.000000
                      ...   
pd_duration         0.000000
NP3FRZGT            0.000000
NP2TRMR             0.000000
NP3RIGRU            0.000000
DYSKIRAT     

In [70]:
print(X_train.columns)

Index(['NP3KTRMR', 'NP1ANXS', 'demopd_ageassess', 'GDS15SCORE', 'LDOPARX',
       'NP1SLPN', 'NP3POSTR', 'NP2DRES', 'NP3RIGRL', 'NP1HALL', 'NP3FTAPR',
       'NP3RIGLU', 'NP1DDS', 'NP3RIGN', 'NP1PAIN', 'NP1APAT', 'NP3RTALL',
       'NP4FLCTX', 'PDCLNSTA', 'NP3RISNG', 'gender', 'NP3BRADY', 'NP3TTAPR',
       'NP4FLCTI', 'NP3PTRMR', 'NP3SPCH', 'NP3PRSPR', 'NP3TTAPL', 'NP2SWAL',
       'NP3RTARL', 'NP3PTRML', 'DYSKPRES', 'NP1SLPD', 'NP4OFF', 'NHY',
       'NP1FATG', 'NP3RTARU', 'NP1LTHD', 'NP3HMOVR', 'NP3PRSPL', 'NP3GAIT',
       'NP3LGAGR', 'NP2HWRT', 'NP1URN', 'MINSNCLD', 'NP1DPRS', 'NP1COG',
       'NP2SPCH', 'NP4WDYSK', 'NP3LGAGL', 'NP2TURN', 'NP1CNST', 'NP3KTRML',
       'NP3RTALU', 'NP3RIGLL', 'NP2RISE', 'NP3RTCON', 'NP2HOBB', 'NP2SALV',
       'NP3FACXP', 'NP2EAT', 'NP4DYSTN', 'NP2WALK', 'NP2FREZ', 'NP2HYGN',
       'NP3HMOVL', 'NP3PSTBL', 'NP3FTAPL', 'NP4DYSKI', 'NP3RTALJ',
       'pd_duration', 'NP3FRZGT', 'NP2TRMR', 'NP3RIGRU', 'DYSKIRAT'],
      dtype='object')


In [71]:
from collections import defaultdict

In [72]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [73]:
# Drop the feature "PDCLNSTA" as it has null values
X_train= X_train.drop(['PDCLNSTA'], axis=1)
X_test= X_test.drop(['PDCLNSTA'], axis=1)

In [74]:
# impute continuous features with median of the respective feature on the training data, and impute categorical features with mode of the respective feature of the training data

imputation_val_dict= defaultdict()

In [75]:
for col_name in X_train.columns:
    if col_name in ['GDS15SCORE', 'MINSNCLD', 'pd_duration', 'demopd_ageassess']:
        #X_train[col_name]= X_train[col_name].astype(int)
        #X_test[col_name]= X_test[col_name].astype(int)

        # we replace with the median value
        #val_list= np.array(X_train[col_name])
        imputation_val_dict[col_name]= X_train[col_name].median()

        # imputating
        #X_train[col_name]= X_train[col_name].fillna()
    else:
        X_train[col_name]= X_train[col_name].astype("category")
        X_test[col_name]= X_test[col_name].astype("category")
        
        # we replace with the mode value
        val_list= list(X_train[col_name])
        l = [x for x in val_list if pd.isnull(x) is False]
        #print(col_name, l)
        #val_list = val_list[~np.isnan(val_list)]
        imputation_val_dict[col_name]= most_frequent(l)


In [76]:
from pprint import pprint
pprint(imputation_val_dict)

defaultdict(None,
            {'DYSKIRAT': 0.0,
             'DYSKPRES': 0,
             'GDS15SCORE': 9.0,
             'LDOPARX': 1,
             'MINSNCLD': 240.0,
             'NHY': 2,
             'NP1ANXS': 0,
             'NP1APAT': 1,
             'NP1CNST': 0.0,
             'NP1COG': 0,
             'NP1DDS': 0,
             'NP1DPRS': 1,
             'NP1FATG': 1.0,
             'NP1HALL': 0,
             'NP1LTHD': 0.0,
             'NP1PAIN': 1.0,
             'NP1SLPD': 0.0,
             'NP1SLPN': 0.0,
             'NP1URN': 0.0,
             'NP2DRES': 1,
             'NP2EAT': 1,
             'NP2FREZ': 0,
             'NP2HOBB': 1,
             'NP2HWRT': 1,
             'NP2HYGN': 1,
             'NP2RISE': 1,
             'NP2SALV': 0,
             'NP2SPCH': 0,
             'NP2SWAL': 0,
             'NP2TRMR': 1,
             'NP2TURN': 1,
             'NP2WALK': 1,
             'NP3BRADY': 2,
             'NP3FACXP': 1,
             'NP3FRZGT': 0,
             '

In [77]:
print(imputation_val_dict['gender'])

2


In [78]:
X_train1= X_train
X_test1= X_test

In [79]:
for col_name in X_train1.columns:
    X_train1[col_name]= X_train1[col_name].fillna(imputation_val_dict[col_name])
    X_test1[col_name]= X_test1[col_name].fillna(imputation_val_dict[col_name])
    

In [ ]:
# valid statement but commented to stop updating data files
X_train1.to_csv('train_data_imputed_ML_classification.csv', index=False)
X_test1.to_csv('test_data_imputed_ML_classification.csv', index=False)
y_train.to_csv('target_label_train_data_imputed_ML_classification.csv', index=False)
y_test.to_csv('target_label_test_data_imputed_ML_classification.csv', index=False)

## Data imputation is complete. Data is ready for decision tree classification model

### Subtypes obtained from minimum number of samples per leaf is set to 10

In [ ]:
# Read mds data, keep only overlapping columns in MDS
mds_data1= pd.read_csv("MDS_added_new_vars_MDS_data.csv")
mds_data1.head()

,patinit,patnum,picode,assessdate,ybirth,age,sex,educat,yeduc,civilst,...,nmstot,mdsupdrs_I,mdsupdrs_II,mdsupdrs_III,mdsupdrs_IV,cisipd,age_c,duratpd_c,epworth_mdsnms,depression_sum
0,MSS,1,3,10/26/2016,1945,71,0,4,20,2,...,125.0,20.0,27.0,44.0,5.0,14.0,2,3,9,7.0
1,TF,10,3,11/28/2016,1948,68,0,4,18,2,...,54.0,5.0,14.0,28.0,2.0,6.0,2,2,0,0.0
2,SP,100,3,12/11/2017,1936,81,0,4,20,2,...,39.0,11.0,14.0,23.0,1.0,9.0,3,3,4,0.0
3,MK,101,3,1/12/2018,1956,61,0,4,17,2,...,105.0,26.0,25.0,29.0,6.0,9.0,2,3,9,4.0
4,LK,102,3,1/16/2018,1954,63,1,4,16,2,...,109.0,23.0,18.0,31.0,5.0,11.0,2,3,6,4.0


In [ ]:
# final training dataset of LRRK2 used
lrrk2_data1= pd.read_csv("final_training_data.csv")

In [83]:
mds_features= list(mapped_features_in_lrrk2.values()) + list(mds_updrs_mapping.values()) + ['patnum']
print(mds_features)


['durat_pd', 'pdonset', 'age', 'sex', 'moca', 'hy', 'ldopadose__LED_', 'epworth_mdsnms', 'depression_sum', 'mdsupdrs1_1', 'mdsupdrs1_2', 'mdsupdrs1_3', 'mdsupdrs1_4', 'mdsupdrs1_5', 'mdsupdrs1_6', 'mdsupdrs1_7', 'mdsupdrs1_8', 'mdsupdrs1_9', 'mdsupdrs1_10', 'mdsupdrs1_11', 'mdsupdrs1_12', 'mdsupdrs1_13', 'mdsupdrs2_1', 'mdsupdrs2_2', 'mdsupdrs2_3', 'mdsupdrs2_4', 'mdsupdrs2_5', 'mdsupdrs2_6', 'mdsupdrs2_7', 'mdsupdrs2_8', 'mdsupdrs2_9', 'mdsupdrs2_10', 'mdsupdrs2_11', 'mdsupdrs2_12', 'mdsupdrs2_13', 'mdsupdrs3a', 'mdsupdrs3b', 'mdsupdrs3c', 'mdsupdrs3c1', 'mdsupdrs3_1', 'mdsupdrs3_2', 'mdsupdrs3_3a', 'mdsupdrs3_3b', 'mdsupdrs3_3c', 'mdsupdrs3_3d', 'mdsupdrs3_3e', 'mdsupdrs3_4a', 'mdsupdrs3_4b', 'mdsupdrs3_5a', 'mdsupdrs3_5b', 'mdsupdrs3_6a', 'mdsupdrs3_6b', 'mdsupdrs3_7a', 'mdsupdrs3_7b', 'mdsupdrs3_8a', 'mdsupdrs3_8b', 'mdsupdrs3_9', 'mdsupdrs3_10', 'mdsupdrs3_11', 'mdsupdrs3_12', 'mdsupdrs3_13', 'mdsupdrs3_14', 'mdsupdrs3_15a', 'mdsupdrs3_15b', 'mdsupdrs3_16a', 'mdsupdrs3_16b', 'mdsu

In [84]:
mds_data2= mds_data1[mds_features]
mds_data2.head()

,durat_pd,pdonset,age,sex,moca,hy,ldopadose__LED_,epworth_mdsnms,depression_sum,mdsupdrs1_1,...,mdsupdrs_dysk,mdsupdrs_int,mdsupdrs_hy,mdsupdrs4_1,mdsupdrs4_2,mdsupdrs4_3,mdsupdrs4_4,mdsupdrs4_5,mdsupdrs4_6,patnum
0,13,58,71,0,25.0,2,600,9,7.0,1,...,0,0,2,1,1,1,2.0,0.0,0,1
1,5,63,68,0,29.0,2,500,0,0.0,0,...,0,0,2,1,0,0,1.0,0.0,0,10
2,15,66,81,0,24.0,2,600,4,0.0,2,...,0,0,2,1,0,0,0.0,0.0,0,100
3,15,46,61,0,23.0,2,700,9,4.0,1,...,0,0,2,0,0,2,3.0,1.0,0,101
4,8,55,63,1,29.0,3,500,6,4.0,1,...,0,0,3,1,0,1,1.0,1.0,1,102


In [85]:
# select early-onset PD patients in MDS dataset
mds_data3= mds_data2[mds_data2['pdonset'] < 50 ]
mds_data3= mds_data3[mds_data3['pdonset'] >= 20 ]
print(mds_data3.shape)

(68, 82)


In [86]:
print(mds_data2.shape)

(402, 82)


In [87]:
# select late-onset PD patients in MDS dataset
mds_data4= mds_data2[mds_data2['pdonset'] > 50 ]
print(mds_data4.shape)

(321, 82)


In [88]:
print(mds_data4.columns)

Index(['durat_pd', 'pdonset', 'age', 'sex', 'moca', 'hy', 'ldopadose__LED_',
       'epworth_mdsnms', 'depression_sum', 'mdsupdrs1_1', 'mdsupdrs1_2',
       'mdsupdrs1_3', 'mdsupdrs1_4', 'mdsupdrs1_5', 'mdsupdrs1_6',
       'mdsupdrs1_7', 'mdsupdrs1_8', 'mdsupdrs1_9', 'mdsupdrs1_10',
       'mdsupdrs1_11', 'mdsupdrs1_12', 'mdsupdrs1_13', 'mdsupdrs2_1',
       'mdsupdrs2_2', 'mdsupdrs2_3', 'mdsupdrs2_4', 'mdsupdrs2_5',
       'mdsupdrs2_6', 'mdsupdrs2_7', 'mdsupdrs2_8', 'mdsupdrs2_9',
       'mdsupdrs2_10', 'mdsupdrs2_11', 'mdsupdrs2_12', 'mdsupdrs2_13',
       'mdsupdrs3a', 'mdsupdrs3b', 'mdsupdrs3c', 'mdsupdrs3c1', 'mdsupdrs3_1',
       'mdsupdrs3_2', 'mdsupdrs3_3a', 'mdsupdrs3_3b', 'mdsupdrs3_3c',
       'mdsupdrs3_3d', 'mdsupdrs3_3e', 'mdsupdrs3_4a', 'mdsupdrs3_4b',
       'mdsupdrs3_5a', 'mdsupdrs3_5b', 'mdsupdrs3_6a', 'mdsupdrs3_6b',
       'mdsupdrs3_7a', 'mdsupdrs3_7b', 'mdsupdrs3_8a', 'mdsupdrs3_8b',
       'mdsupdrs3_9', 'mdsupdrs3_10', 'mdsupdrs3_11', 'mdsupdrs3_12',
       '

In [89]:
print(mds_data3.columns)

Index(['durat_pd', 'pdonset', 'age', 'sex', 'moca', 'hy', 'ldopadose__LED_',
       'epworth_mdsnms', 'depression_sum', 'mdsupdrs1_1', 'mdsupdrs1_2',
       'mdsupdrs1_3', 'mdsupdrs1_4', 'mdsupdrs1_5', 'mdsupdrs1_6',
       'mdsupdrs1_7', 'mdsupdrs1_8', 'mdsupdrs1_9', 'mdsupdrs1_10',
       'mdsupdrs1_11', 'mdsupdrs1_12', 'mdsupdrs1_13', 'mdsupdrs2_1',
       'mdsupdrs2_2', 'mdsupdrs2_3', 'mdsupdrs2_4', 'mdsupdrs2_5',
       'mdsupdrs2_6', 'mdsupdrs2_7', 'mdsupdrs2_8', 'mdsupdrs2_9',
       'mdsupdrs2_10', 'mdsupdrs2_11', 'mdsupdrs2_12', 'mdsupdrs2_13',
       'mdsupdrs3a', 'mdsupdrs3b', 'mdsupdrs3c', 'mdsupdrs3c1', 'mdsupdrs3_1',
       'mdsupdrs3_2', 'mdsupdrs3_3a', 'mdsupdrs3_3b', 'mdsupdrs3_3c',
       'mdsupdrs3_3d', 'mdsupdrs3_3e', 'mdsupdrs3_4a', 'mdsupdrs3_4b',
       'mdsupdrs3_5a', 'mdsupdrs3_5b', 'mdsupdrs3_6a', 'mdsupdrs3_6b',
       'mdsupdrs3_7a', 'mdsupdrs3_7b', 'mdsupdrs3_8a', 'mdsupdrs3_8b',
       'mdsupdrs3_9', 'mdsupdrs3_10', 'mdsupdrs3_11', 'mdsupdrs3_12',
       '

## Feature description

mds_data2 - full MDS dataset
mds_data3 = early-onset MDS - 68 patients
mds_data4 = late-onset MDS - 

In [90]:
from collections import defaultdict

In [91]:
mds_col_name_to_id= defaultdict(int)

In [92]:
mds_data3_columns= list(mds_data3.columns)
for elem_id in range(len(mds_data3_columns)):
    mds_col_name_to_id[mds_data3_columns[elem_id]]= elem_id

In [93]:
print(mds_col_name_to_id)

defaultdict(<class 'int'>, {'durat_pd': 0, 'pdonset': 1, 'age': 2, 'sex': 3, 'moca': 4, 'hy': 5, 'ldopadose__LED_': 6, 'epworth_mdsnms': 7, 'depression_sum': 8, 'mdsupdrs1_1': 9, 'mdsupdrs1_2': 10, 'mdsupdrs1_3': 11, 'mdsupdrs1_4': 12, 'mdsupdrs1_5': 13, 'mdsupdrs1_6': 14, 'mdsupdrs1_7': 15, 'mdsupdrs1_8': 16, 'mdsupdrs1_9': 17, 'mdsupdrs1_10': 18, 'mdsupdrs1_11': 19, 'mdsupdrs1_12': 20, 'mdsupdrs1_13': 21, 'mdsupdrs2_1': 22, 'mdsupdrs2_2': 23, 'mdsupdrs2_3': 24, 'mdsupdrs2_4': 25, 'mdsupdrs2_5': 26, 'mdsupdrs2_6': 27, 'mdsupdrs2_7': 28, 'mdsupdrs2_8': 29, 'mdsupdrs2_9': 30, 'mdsupdrs2_10': 31, 'mdsupdrs2_11': 32, 'mdsupdrs2_12': 33, 'mdsupdrs2_13': 34, 'mdsupdrs3a': 35, 'mdsupdrs3b': 36, 'mdsupdrs3c': 37, 'mdsupdrs3c1': 38, 'mdsupdrs3_1': 39, 'mdsupdrs3_2': 40, 'mdsupdrs3_3a': 41, 'mdsupdrs3_3b': 42, 'mdsupdrs3_3c': 43, 'mdsupdrs3_3d': 44, 'mdsupdrs3_3e': 45, 'mdsupdrs3_4a': 46, 'mdsupdrs3_4b': 47, 'mdsupdrs3_5a': 48, 'mdsupdrs3_5b': 49, 'mdsupdrs3_6a': 50, 'mdsupdrs3_6b': 51, 'mdsupd

In [94]:
print(mds_updrs_mapping['NP4WDYSK'], mds_updrs_mapping['NP3FTAPL'], mds_updrs_mapping['NP4FLCTI'], mds_updrs_mapping['NP3HMOVR'])

mdsupdrs4_1 mdsupdrs3_4b mdsupdrs4_4 mdsupdrs3_5a


In [95]:
print(mds_updrs_mapping['NP1CNST'], mds_updrs_mapping['NP3RTARU'], mds_updrs_mapping['NP3GAIT'], mds_updrs_mapping['NP3RIGN'])

mdsupdrs1_11 mdsupdrs3_17a mdsupdrs3_10 mdsupdrs3_3a


In [96]:
print(Counter(mds_data3['sex']))

Counter({0: 42, 1: 26})


In [97]:
patient_mapped_to_subtype_id= defaultdict(list)

In [98]:
for row_id in range(mds_data3.shape[0]):
    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] != 0:
        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_4b']] <= 1:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs4_4']] >= 3:
                patient_mapped_to_subtype_id['1'].append(mds_data3.iloc[row_id, 81])
            else:
                if mds_data3.iloc[row_id, 3] == 0: # male
                    patient_mapped_to_subtype_id['2'].append(mds_data3.iloc[row_id, 81])
        else:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_5a']] >= 3:
                patient_mapped_to_subtype_id['3'].append(mds_data3.iloc[row_id, 81])
    else:
        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0 and mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] == 0:
                patient_mapped_to_subtype_id['4'].append(mds_data3.iloc[row_id, 81])
            elif mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] <= 2:
                if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_3a']] <= 2:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['depression_sum']] >= 15:
                        patient_mapped_to_subtype_id['5'].append(mds_data3.iloc[row_id, 81])
                else:
                    patient_mapped_to_subtype_id['6'].append(mds_data3.iloc[row_id, 81])
    

In [99]:
print(patient_mapped_to_subtype_id)

defaultdict(<class 'list'>, {'4': ['105', '27', '40', '83', 'KCH-002', 'KCH-005', 'KCH-012', 'KCH-013', 'KCH-020', 'KCH-039', 'KCH-046', 'KCH-098', 'KCH-121', 'KCH-127', 'KCH-132', 'LDH026', 'LIS03', 'RFL051', 'RFL059', 'RFL060'], '3': ['116', 'KCH-134', 'LDH040'], '1': ['166', '57'], '2': ['53', 'KCH-100', 'KCH-123']})


In [100]:
for key_id in patient_mapped_to_subtype_id.keys():
    print(key_id, len(set(patient_mapped_to_subtype_id[key_id])))

4 20
3 3
1 2
2 3


In [101]:
# early onset PD subtypes
subtype1_dict= {'NP4WDYSK': [1, 4], 'NP3FTAPL': [0, 1], 'NP4FLCTI': [3, 4]}
subtype2_dict= {'NP4WDYSK': [1, 4], 'NP3FTAPL': [0, 1], 'NP4FLCTI': [0, 2], 'gender': [2,2]}
subtype3_dict= {'NP4WDYSK': [1, 4], 'NP3FTAPL': [2, 4], 'NP3HMOVR': [3, 4]}
subtype4_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [0, 0], 'NP3GAIT': [0, 1]}
subtype5_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [1, 2], 'NP3GAIT': [0, 1], 'NP3RIGN': [0, 2], 'GDS15SCORE': [9, 15]}
subtype6_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [1, 2], 'NP3GAIT': [0, 1], 'NP3RIGN': [3, 4]}


In [102]:
# late onset PD subtypes
subtype1_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [1, 2], 'NP3GAIT': [0, 1], 'NP3RIGN': [0, 2], 'GDS15SCORE': [0, 8]}
subtype2_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [0, 2], 'NP3GAIT': [2, 4], 'NP3RTARL': [0, 0]}
subtype3_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [0, 0], 'NP3RTARU': [3, 4]}
subtype4_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [1, 4], 'NP3TTAPR': [0, 3], 'MINSNCLD': [0, 232], 'NP3GAIT': [2, 4]}
subtype5_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [1, 4], 'NP3TTAPR': [0, 3], 'MINSNCLD': [233, ], 'NP3POSTR': [2, 4]}
subtype6_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [1, 4], 'NP3TTAPR': [0, 3], 'MINSNCLD': [233, ], 'NP3POSTR': [0, 1], 'NP2TRMR': [0, 2], 'NP3GAIT': [0, 1]}
subtype7_late_dict= {'NP4WDYSK': [0, 0], 'NP1CNST': [1, 4], 'NP3TTAPR': [0, 3], 'MINSNCLD': [233, ], 'NP3POSTR': [0, 1], 'NP2TRMR': [0, 2], 'NP3GAIT': [2, 4], 'NP3RTCON': [2, 4]}


In [103]:
print(mds_updrs_mapping['NP3RTARL'], mds_updrs_mapping['NP3TTAPR'], mds_updrs_mapping['MINSNCLD'], mds_updrs_mapping['NP3POSTR'])

mdsupdrs3_17c mdsupdrs3_7a mdsupdrs3c1 mdsupdrs3_13


In [104]:
print(mds_updrs_mapping['NP2TRMR'], mds_updrs_mapping['NP3RTCON'])

mdsupdrs2_10 mdsupdrs3_18


In [105]:
print(mds_updrs_mapping['NP4WDYSK'], mds_updrs_mapping['NP3FTAPL'], mds_updrs_mapping['NP4FLCTI'], mds_updrs_mapping['NP3HMOVR'])

mdsupdrs4_1 mdsupdrs3_4b mdsupdrs4_4 mdsupdrs3_5a


In [106]:
print(mds_updrs_mapping['NP1CNST'], mds_updrs_mapping['NP3RTARU'], mds_updrs_mapping['NP3GAIT'], mds_updrs_mapping['NP3RIGN'])

mdsupdrs1_11 mdsupdrs3_17a mdsupdrs3_10 mdsupdrs3_3a


In [107]:
# late-onset subtype conditions mapped to early-onset PD patients of MDS dataset
patient_late_mapped_to_subtype_id= defaultdict(list)

In [108]:
for row_id in range(mds_data3.shape[0]):
    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] == 0:
        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] > 2:
                patient_late_mapped_to_subtype_id['3'].append(mds_data3.iloc[row_id, 81])
            else:
                if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] > 0 and mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_3a']] <= 2 and mds_data3.iloc[row_id, mds_col_name_to_id['depression_sum']]  < 15:
                        patient_late_mapped_to_subtype_id['1'].append(mds_data3.iloc[row_id, 81])
                else:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17c']] == 0:
                        patient_late_mapped_to_subtype_id['2'].append(mds_data3.iloc[row_id, 81])
        else:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_7a']] <= 3:
                if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3c1']] > 232:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_13']] >= 2:
                        patient_late_mapped_to_subtype_id['5'].append(mds_data3.iloc[row_id, 81])
                    else:
                        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs2_10']] <= 2:
                            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                                patient_late_mapped_to_subtype_id['6'].append(mds_data3.iloc[row_id, 81])
                            else:
                                if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_18']] >= 2:
                                    patient_late_mapped_to_subtype_id['7'].append(mds_data3.iloc[row_id, 81])
                else:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2:
                        patient_late_mapped_to_subtype_id['4'].append(mds_data3.iloc[row_id, 81])

In [109]:
print(patient_late_mapped_to_subtype_id)

defaultdict(<class 'list'>, {'4': ['107', 'KCH-049', 'KCH-085', 'LDH004'], '2': ['159'], '1': ['52', '99', 'KCH-045', 'KCH-088', 'KCH-097', 'KCH-116'], '6': ['KCH-047', 'KCH-086']})


In [110]:
# late-onset subtype conditions mapped to late-onset PD patients of MDS dataset
patient_late_mapped_to_subtype_id_lateonset_mds= defaultdict(list)

In [111]:
for row_id in range(mds_data4.shape[0]):
    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] == 0:
        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] > 2:
                patient_late_mapped_to_subtype_id_lateonset_mds['3'].append(mds_data4.iloc[row_id, 81])
            else:
                if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] > 0 and mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_3a']] <= 2 and mds_data4.iloc[row_id, mds_col_name_to_id['depression_sum']]  < 15:
                        patient_late_mapped_to_subtype_id_lateonset_mds['1'].append(mds_data4.iloc[row_id, 81])
                else:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17c']] == 0:
                        patient_late_mapped_to_subtype_id_lateonset_mds['2'].append(mds_data4.iloc[row_id, 81])
        else:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_7a']] <= 3:
                if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3c1']] > 232:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_13']] >= 2:
                        patient_late_mapped_to_subtype_id_lateonset_mds['5'].append(mds_data4.iloc[row_id, 81])
                    else:
                        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs2_10']] <= 2:
                            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                                patient_late_mapped_to_subtype_id_lateonset_mds['6'].append(mds_data4.iloc[row_id, 81])
                            else:
                                if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_18']] >= 2:
                                    patient_late_mapped_to_subtype_id_lateonset_mds['7'].append(mds_data4.iloc[row_id, 81])
                else:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2:
                        patient_late_mapped_to_subtype_id_lateonset_mds['4'].append(mds_data4.iloc[row_id, 81])

In [112]:
print(patient_late_mapped_to_subtype_id_lateonset_mds)

defaultdict(<class 'list'>, {'1': ['106', '140', '147', '164', '165', '3', '32', '33', '37', '56', '60', '67', '8', '88', 'KCH-004', 'KCH-008', 'KCH-018', 'KCH-031', 'KCH-038', 'KCH-042', 'KCH-062', 'KCH-075', 'KCH-092', 'KCH-101', 'KCH-111', 'KCH-119', 'LDH005', 'LDH041', 'LDH042', 'LDH048', 'LEW055', 'LEW060', 'LEW062', 'LIS05', 'LIS13', 'LIS21', 'LIS24', 'RFL053', 'RFL062'], '6': ['108', '113', '138', '20', 'KCH-080', 'KCH-089', 'LDH002', 'LEW039', 'LIS06', 'LIS09', 'LIS12', 'RFL048'], '3': ['111', '122', '91', '93', 'KCH-014', 'KCH-055', 'LEW051'], '5': ['120', '31', 'KCH-087', 'KCH-090', 'KCH-126', 'LDH014', 'LEW049'], '4': ['132', '161', '21', '82', 'KCH-026', 'KCH-032', 'KCH-033', 'KCH-034', 'KCH-037', 'KCH-051', 'KCH-052', 'KCH-053', 'KCH-054', 'KCH-131', 'LDH018', 'LDH027', 'LDH028', 'LDH029', 'LDH036', 'LIS11', 'LIS17', 'LIS23', 'RFL045', 'RFL056', 'RFL057'], '2': ['137', '169', '17', '34', '38', '49', '81', '89', 'KCH-015', 'KCH-027', 'KCH-067', 'KCH-082', 'LDH001', 'LDH003'

In [113]:
for key_id in patient_late_mapped_to_subtype_id_lateonset_mds.keys():
    print(key_id, len(set(patient_late_mapped_to_subtype_id_lateonset_mds[key_id])))

1 39
6 12
3 7
5 7
4 25
2 29


In [114]:
# early-onset subtype conditions mapped to late-onset PD patients of MDS dataset
patient_early_mapped_to_subtype_id_lateonset_mds= defaultdict(list)

In [115]:
for row_id in range(mds_data4.shape[0]):
    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] != 0:
        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_4b']] <= 1:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs4_4']] >= 3:
                patient_early_mapped_to_subtype_id_lateonset_mds['1'].append(mds_data4.iloc[row_id, 81])
            else:
                if mds_data4.iloc[row_id, 3] == 0: # male
                    patient_early_mapped_to_subtype_id_lateonset_mds['2'].append(mds_data4.iloc[row_id, 81])
        else:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_5a']] >= 3:
                patient_early_mapped_to_subtype_id_lateonset_mds['3'].append(mds_data4.iloc[row_id, 81])
    else:
        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0 and mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] == 0:
                patient_early_mapped_to_subtype_id_lateonset_mds['4'].append(mds_data4.iloc[row_id, 81])
            elif mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] <= 2:
                if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_3a']] <= 2:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['depression_sum']] >= 15:
                        patient_early_mapped_to_subtype_id_lateonset_mds['5'].append(mds_data4.iloc[row_id, 81])
                else:
                    patient_early_mapped_to_subtype_id_lateonset_mds['6'].append(mds_data4.iloc[row_id, 81])

In [116]:
print(patient_early_mapped_to_subtype_id_lateonset_mds)

defaultdict(<class 'list'>, {'2': ['1', '10', '100', '117', '150', '151', '18', '22', '39', '41', '45', '74', '76', '94', '95', '96', 'KCH-058', 'KCH-070', 'KCH-077', 'KCH-091', 'KCH-105', 'KCH-130', 'LDH038'], '1': ['103', '112', '129', '75', '77', '78', 'KCH-083', 'KCH-103', 'KCH-113'], '4': ['11', '110', '12', '126', '128', '131', '133', '136', '14', '153', '24', '25', '26', '36', '54', '59', '61', '70', '79', '80', 'KCH-001', 'KCH-006', 'KCH-007', 'KCH-022', 'KCH-035', 'KCH-056', 'KCH-060', 'KCH-061', 'KCH-065', 'KCH-069', 'KCH-072', 'KCH-073', 'KCH-078', 'KCH-081', 'KCH-094', 'KCH-118', 'LDH009', 'LDH021', 'LDH022', 'LDH025', 'LDH031', 'LDH049', 'LDH057', 'LEW045', 'LEW064', 'LIS01', 'LIS04', 'LIS19', 'LIS22', 'RFL050'], '3': ['163', '42', '51', '84', '92'], '5': ['LDH010', 'LDH012', 'LEW056', 'LEW057'], '6': ['LIS14']})


In [117]:
for key_id in patient_early_mapped_to_subtype_id_lateonset_mds.keys():
    print(key_id, len(patient_early_mapped_to_subtype_id_lateonset_mds[key_id]))

2 23
1 9
4 50
3 5
5 4
6 1


## Mixed subtype mapping

In [118]:
patient_early_mapped_to_subtype_id_mixed_mds= defaultdict(list)

In [119]:
print(mds_updrs_mapping['NP3RTARL'], mds_updrs_mapping['NP3TTAPR'], mds_updrs_mapping['MINSNCLD'], mds_updrs_mapping['NP3POSTR'])
print(mds_updrs_mapping['NP2TRMR'], mds_updrs_mapping['NP3RTCON'])
print(mds_updrs_mapping['NP4WDYSK'], mds_updrs_mapping['NP3FTAPL'], mds_updrs_mapping['NP4FLCTI'], mds_updrs_mapping['NP3HMOVR'])
print(mds_updrs_mapping['NP1CNST'], mds_updrs_mapping['NP3RTARU'], mds_updrs_mapping['NP3GAIT'], mds_updrs_mapping['NP3RIGN'])

mdsupdrs3_17c mdsupdrs3_7a mdsupdrs3c1 mdsupdrs3_13
mdsupdrs2_10 mdsupdrs3_18
mdsupdrs4_1 mdsupdrs3_4b mdsupdrs4_4 mdsupdrs3_5a
mdsupdrs1_11 mdsupdrs3_17a mdsupdrs3_10 mdsupdrs3_3a


### mds_data2 is complete MDS dataset of 402 patients

In [120]:
for row_id in range(mds_data2.shape[0]):
    if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] == 0:
        if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0:
            if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] <= 2 and mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17c']] >= 1:
                patient_early_mapped_to_subtype_id_mixed_mds['1'].append(mds_data2.iloc[row_id, 81])
        else:
            if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_7a']] == 4:
                patient_early_mapped_to_subtype_id_mixed_mds['2'].append(mds_data2.iloc[row_id, 81])
            else:
                if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3c1']] <= 232:
                    if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                        patient_early_mapped_to_subtype_id_mixed_mds['3'].append(mds_data2.iloc[row_id, 81])
                else:
                    if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_13']] <= 1:
                        if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs2_10']] >= 3:
                            patient_early_mapped_to_subtype_id_mixed_mds['4'].append(mds_data2.iloc[row_id, 81])
                        else:
                            if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_18']] <= 1:
                                patient_early_mapped_to_subtype_id_mixed_mds['5'].append(mds_data2.iloc[row_id, 81])
    else:
        if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_4b']] <= 1:
            if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs4_4']] <= 2 and mds_data2.iloc[row_id, 3] == 1:
                patient_early_mapped_to_subtype_id_mixed_mds['6'].append(mds_data2.iloc[row_id, 81])
        else:
            if mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_5a']] <= 2:
                patient_early_mapped_to_subtype_id_mixed_mds['7'].append(mds_data2.iloc[row_id, 81])

In [121]:
print(patient_early_mapped_to_subtype_id_mixed_mds)

defaultdict(<class 'list'>, {'3': ['101', '104', '115', '125', '127', '13', '134', '139', '15', '19', '2', '35', '4', '43', '64', '7', '71', 'KCH-010', 'KCH-017', 'KCH-024', 'KCH-030', 'KCH-036', 'KCH-043', 'KCH-044', 'KCH-057', 'KCH-064', 'KCH-068', 'KCH-071', 'KCH-095', 'KCH-109', 'KCH-110', 'KCH-112', 'KCH-136', 'LDH032', 'LDH034', 'LDH035', 'LDH044', 'LDH046', 'LDH053', 'LDH056', 'LEW040', 'LIS07', 'LIS08', 'RFL046', 'RFL052', 'RFL061', 'RFL063'], '6': ['102', '142', '155', '16', '28', '44', '50', '6', '69', '86', '87', 'KCH-021', 'KCH-079', 'KCH-124', 'LDH024', 'LEW046'], '5': ['107', '48', 'KCH-051', 'KCH-053', 'LEW059'], '7': ['123', '135', '141', '144', '148', '152', '154', '157', '158', '160', '162', '168', '171', '23', '29', '30', '46', '47', '55', '63', '90', '97', 'KCH-048', 'KCH-084', 'KCH-099', 'KCH-102', 'KCH-104', 'KCH-128', 'KCH-129', 'KCH-135', 'LDH007', 'LDH016', 'LDH017', 'LDH043', 'LDH047', 'LDH054', 'LIS15', 'RFL055'], '1': ['85', 'KCH-019', 'KCH-063', 'LDH006', '

In [122]:
for key_id in patient_early_mapped_to_subtype_id_mixed_mds.keys():
    print(key_id, len(patient_early_mapped_to_subtype_id_mixed_mds[key_id]))

3 47
6 16
5 5
7 38
1 8
2 2
4 2


In [123]:
## Mixed subtypes mapped to late-onset
patient_mixedonset_mapped_to_subtype_id_lateonset_mds= defaultdict(list)

In [124]:
for row_id in range(mds_data4.shape[0]):
    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] == 0:
        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] <= 2 and mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data2.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17c']] >= 1:
                patient_mixedonset_mapped_to_subtype_id_lateonset_mds['1'].append(mds_data4.iloc[row_id, 81])
        else:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_7a']] == 4:
                patient_mixedonset_mapped_to_subtype_id_lateonset_mds['2'].append(mds_data4.iloc[row_id, 81])
            else:
                if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3c1']] <= 232:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                        patient_mixedonset_mapped_to_subtype_id_lateonset_mds['3'].append(mds_data4.iloc[row_id, 81])
                else:
                    if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_13']] <= 1:
                        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs2_10']] >= 3:
                            patient_mixedonset_mapped_to_subtype_id_lateonset_mds['4'].append(mds_data4.iloc[row_id, 81])
                        else:
                            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_18']] <= 1:
                                patient_mixedonset_mapped_to_subtype_id_lateonset_mds['5'].append(mds_data4.iloc[row_id, 81])
    else:
        if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_4b']] <= 1:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs4_4']] <= 2 and mds_data4.iloc[row_id, 3] == 1:
                patient_mixedonset_mapped_to_subtype_id_lateonset_mds['6'].append(mds_data4.iloc[row_id, 81])
        else:
            if mds_data4.iloc[row_id, mds_col_name_to_id['mdsupdrs3_5a']] <= 2:
                patient_mixedonset_mapped_to_subtype_id_lateonset_mds['7'].append(mds_data4.iloc[row_id, 81])

In [125]:
print(patient_mixedonset_mapped_to_subtype_id_lateonset_mds)

defaultdict(<class 'list'>, {'6': ['102', '142', '16', '28', '44', '50', '6', '69', '87', 'KCH-021', 'KCH-079', 'KCH-124', 'LEW046'], '3': ['104', '115', '125', '127', '13', '134', '139', '15', '19', '2', '35', '43', '64', '7', '71', 'KCH-017', 'KCH-024', 'KCH-030', 'KCH-036', 'KCH-043', 'KCH-044', 'KCH-057', 'KCH-068', 'KCH-095', 'KCH-109', 'KCH-136', 'LDH032', 'LDH034', 'LDH035', 'LDH044', 'LDH046', 'LDH056', 'LEW040', 'LIS08', 'RFL046', 'RFL052', 'RFL061', 'RFL063'], '7': ['123', '135', '144', '148', '152', '157', '160', '162', '30', '46', '47', '55', '63', '90', 'KCH-084', 'KCH-102', 'LDH016', 'LDH017', 'LDH043', 'LDH047', 'LDH054', 'RFL055'], '5': ['48', 'KCH-051', 'KCH-053', 'LEW059'], '1': ['81', '85', 'KCH-063', 'LDH006', 'LIS10'], '2': ['LDH013', 'LDH015'], '4': ['LEW043', 'LEW044']})


In [126]:
{'6': ['102', '142', '16', '28', '44', '50', '6', '69', '87', 'KCH-021', 'KCH-079', 'KCH-124', 'LEW046'], '3': ['104', '115', '125', '127', '13', '134', '139', '15', '19', '2', '35', '43', '64', '7', '71', 'KCH-017', 'KCH-024', 'KCH-030', 'KCH-036', 'KCH-043', 'KCH-044', 'KCH-057', 'KCH-068', 'KCH-095', 'KCH-109', 'KCH-136', 'LDH032', 'LDH034', 'LDH035', 'LDH044', 'LDH046', 'LDH056', 'LEW040', 'LIS08', 'RFL046', 'RFL052', 'RFL061', 'RFL063'], '7': ['123', '135', '144', '148', '152', '157', '160', '162', '30', '46', '47', '55', '63', '90', 'KCH-084', 'KCH-102', 'LDH016', 'LDH017', 'LDH043', 'LDH047', 'LDH054', 'RFL055'], '5': ['48', 'KCH-051', 'KCH-053', 'LEW059'], '1': ['81', '85', 'KCH-063', 'LDH006', 'LIS10'], '2': ['LDH013', 'LDH015'], '4': ['LEW043', 'LEW044']}

{'6': ['102',
  '142',
  '16',
  '28',
  '44',
  '50',
  '6',
  '69',
  '87',
  'KCH-021',
  'KCH-079',
  'KCH-124',
  'LEW046'],
 '3': ['104',
  '115',
  '125',
  '127',
  '13',
  '134',
  '139',
  '15',
  '19',
  '2',
  '35',
  '43',
  '64',
  '7',
  '71',
  'KCH-017',
  'KCH-024',
  'KCH-030',
  'KCH-036',
  'KCH-043',
  'KCH-044',
  'KCH-057',
  'KCH-068',
  'KCH-095',
  'KCH-109',
  'KCH-136',
  'LDH032',
  'LDH034',
  'LDH035',
  'LDH044',
  'LDH046',
  'LDH056',
  'LEW040',
  'LIS08',
  'RFL046',
  'RFL052',
  'RFL061',
  'RFL063'],
 '7': ['123',
  '135',
  '144',
  '148',
  '152',
  '157',
  '160',
  '162',
  '30',
  '46',
  '47',
  '55',
  '63',
  '90',
  'KCH-084',
  'KCH-102',
  'LDH016',
  'LDH017',
  'LDH043',
  'LDH047',
  'LDH054',
  'RFL055'],
 '5': ['48', 'KCH-051', 'KCH-053', 'LEW059'],
 '1': ['81', '85', 'KCH-063', 'LDH006', 'LIS10'],
 '2': ['LDH013', 'LDH015'],
 '4': ['LEW043', 'LEW044']}

In [127]:
for key_id in patient_mixedonset_mapped_to_subtype_id_lateonset_mds.keys():
    print(key_id, len(patient_mixedonset_mapped_to_subtype_id_lateonset_mds[key_id]))

6 13
3 38
7 22
5 4
1 5
2 2
4 2


### Combined statistics for all subtypes mapped to MDS data - 6 early onset, 7 late onset and 7 late onset types - all are validated

In [128]:
all_mapped_list= []

In [129]:
for dict_elem in [patient_early_mapped_to_subtype_id_mixed_mds, patient_early_mapped_to_subtype_id_lateonset_mds, patient_late_mapped_to_subtype_id_lateonset_mds, patient_late_mapped_to_subtype_id, patient_mapped_to_subtype_id]:
    for key_within_dict in dict_elem.keys():
        all_mapped_list.extend(dict_elem[key_within_dict])

In [130]:
print(len(set(all_mapped_list)))

367


In [131]:
mapped_successfully= set(mds_data2['patnum']).intersection(set(all_mapped_list))

In [132]:
print(len(mapped_successfully))

367


In [133]:
print(mapped_successfully)

{'36', 'LIS15', '81', '25', 'KCH-020', '15', 'LIS09', 'LEW045', 'LDH012', 'LDH042', 'KCH-084', 'KCH-119', 'KCH-083', 'LDH035', '113', 'LDH013', 'LIS17', 'LDH001', '105', 'LIS24', 'KCH-034', 'LIS04', '171', 'LDH057', '169', '117', 'KCH-042', 'LDH021', 'KCH-055', 'KCH-123', 'LEW040', '107', '29', 'LDH019', 'LDH043', 'RFL061', 'KCH-032', '125', 'RFL056', 'LIS21', 'LDH050', '110', 'LEW043', 'KCH-102', 'KCH-126', '30', 'LEW056', '161', 'KCH-056', 'KCH-095', 'KCH-118', 'LDH022', 'KCH-113', '87', 'LDH010', 'KCH-017', '89', 'KCH-088', 'KCH-099', 'KCH-047', '16', '42', '19', 'LDH046', 'LDH049', '24', '80', '151', 'LDH048', 'KCH-098', '103', '91', 'KCH-060', '164', 'KCH-116', '7', '85', '116', 'KCH-121', '111', 'KCH-026', 'LDH029', 'LDH020', 'KCH-048', 'KCH-109', '88', '99', '141', 'KCH-035', '70', 'RFL048', 'LEW064', '95', 'RFL049', 'KCH-110', '14', 'KCH-021', '162', '67', 'LDH032', 'KCH-094', 'RFL047', 'LEW062', '61', 'RFL059', '57', 'LIS08', 'LIS02', '83', 'RFL055', 'LDH024', 'LDH053', 'LDH00

In [134]:
print(mds_data2.shape)

(402, 82)


In [135]:
367/402

0.9129353233830846

In [ ]:
# if we only focus on early-onset PD patients

In [136]:
patient_mixed_lrrk2_mapped_to_subtype_id_early_mds= defaultdict(list)

In [137]:
for row_id in range(mds_data3.shape[0]):
    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs4_1']] == 0:
        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs1_11']] == 0:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17a']] <= 2 and mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_17c']] >= 1:
                patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['1'].append(mds_data3.iloc[row_id, 81])
        else:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_7a']] == 4:
                patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['2'].append(mds_data3.iloc[row_id, 81])
            else:
                if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3c1']] <= 232:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] <= 1:
                        patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['3'].append(mds_data3.iloc[row_id, 81])
                else:
                    if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_13']] <= 1:
                        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs2_10']] >= 3:
                            patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['4'].append(mds_data3.iloc[row_id, 81])
                        else:
                            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_10']] >= 2 and mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_18']] <= 1:
                                patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['5'].append(mds_data3.iloc[row_id, 81])
    else:
        if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_4b']] <= 1:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs4_4']] <= 2 and mds_data3.iloc[row_id, 3] == 1:
                patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['6'].append(mds_data3.iloc[row_id, 81])
        else:
            if mds_data3.iloc[row_id, mds_col_name_to_id['mdsupdrs3_5a']] <= 2:
                patient_mixed_lrrk2_mapped_to_subtype_id_early_mds['7'].append(mds_data3.iloc[row_id, 81])

In [138]:
print(patient_mixed_lrrk2_mapped_to_subtype_id_early_mds)

defaultdict(<class 'list'>, {'3': ['101', '4', 'KCH-010', 'KCH-071', 'KCH-110', 'KCH-112', 'LIS07'], '5': ['107'], '7': ['141', '154', '158', '168', '171', '97', 'KCH-099', 'KCH-104', 'KCH-128', 'KCH-129', 'KCH-135', 'LIS15'], '6': ['155', '86']})


In [139]:
all_mapped_list_early= []
for dict_elem in [patient_mixed_lrrk2_mapped_to_subtype_id_early_mds, patient_mapped_to_subtype_id]:
    for key_within_dict in dict_elem.keys():
        all_mapped_list_early.extend(dict_elem[key_within_dict])

In [140]:
print(len(set(all_mapped_list_early)))

50


In [141]:
mapped_successfully_early_mixed_lrrk2_with_early_mds= set(mds_data3['patnum']).intersection(set(all_mapped_list_early))

In [142]:
print(len(mapped_successfully_early_mixed_lrrk2_with_early_mds))

50


In [143]:
print(mds_data3.shape)

(68, 82)


In [144]:
50/68

0.7352941176470589

# Final subtypes

In [155]:
# Subtype E4
print(patient_mapped_to_subtype_id['4'])

['105', '27', '40', '83', 'KCH-002', 'KCH-005', 'KCH-012', 'KCH-013', 'KCH-020', 'KCH-039', 'KCH-046', 'KCH-098', 'KCH-121', 'KCH-127', 'KCH-132', 'LDH026', 'LIS03', 'RFL051', 'RFL059', 'RFL060']


In [146]:
print(len(patient_mapped_to_subtype_id['4']))

20


In [150]:
# Late subtypes

print(len(patient_late_mapped_to_subtype_id_lateonset_mds['1']), patient_late_mapped_to_subtype_id_lateonset_mds['1'])

39 ['106', '140', '147', '164', '165', '3', '32', '33', '37', '56', '60', '67', '8', '88', 'KCH-004', 'KCH-008', 'KCH-018', 'KCH-031', 'KCH-038', 'KCH-042', 'KCH-062', 'KCH-075', 'KCH-092', 'KCH-101', 'KCH-111', 'KCH-119', 'LDH005', 'LDH041', 'LDH042', 'LDH048', 'LEW055', 'LEW060', 'LEW062', 'LIS05', 'LIS13', 'LIS21', 'LIS24', 'RFL053', 'RFL062']


In [151]:
print(len(patient_late_mapped_to_subtype_id_lateonset_mds['2']), patient_late_mapped_to_subtype_id_lateonset_mds['2'])

29 ['137', '169', '17', '34', '38', '49', '81', '89', 'KCH-015', 'KCH-027', 'KCH-067', 'KCH-082', 'LDH001', 'LDH003', 'LDH008', 'LDH019', 'LDH020', 'LDH030', 'LDH050', 'LDH052', 'LEW042', 'LEW061', 'LEW065', 'LIS02', 'LIS16', 'LIS25', 'RFL047', 'RFL049', 'RFL054']


In [152]:
print(len(patient_late_mapped_to_subtype_id_lateonset_mds['4']), patient_late_mapped_to_subtype_id_lateonset_mds['4'])

25 ['132', '161', '21', '82', 'KCH-026', 'KCH-032', 'KCH-033', 'KCH-034', 'KCH-037', 'KCH-051', 'KCH-052', 'KCH-053', 'KCH-054', 'KCH-131', 'LDH018', 'LDH027', 'LDH028', 'LDH029', 'LDH036', 'LIS11', 'LIS17', 'LIS23', 'RFL045', 'RFL056', 'RFL057']


In [153]:
# Mixed onset subtypes

print(len(patient_early_mapped_to_subtype_id_mixed_mds['3']), patient_early_mapped_to_subtype_id_mixed_mds['3'])

47 ['101', '104', '115', '125', '127', '13', '134', '139', '15', '19', '2', '35', '4', '43', '64', '7', '71', 'KCH-010', 'KCH-017', 'KCH-024', 'KCH-030', 'KCH-036', 'KCH-043', 'KCH-044', 'KCH-057', 'KCH-064', 'KCH-068', 'KCH-071', 'KCH-095', 'KCH-109', 'KCH-110', 'KCH-112', 'KCH-136', 'LDH032', 'LDH034', 'LDH035', 'LDH044', 'LDH046', 'LDH053', 'LDH056', 'LEW040', 'LIS07', 'LIS08', 'RFL046', 'RFL052', 'RFL061', 'RFL063']


In [154]:
print(len(patient_early_mapped_to_subtype_id_mixed_mds['7']), patient_early_mapped_to_subtype_id_mixed_mds['7'])

38 ['123', '135', '141', '144', '148', '152', '154', '157', '158', '160', '162', '168', '171', '23', '29', '30', '46', '47', '55', '63', '90', '97', 'KCH-048', 'KCH-084', 'KCH-099', 'KCH-102', 'KCH-104', 'KCH-128', 'KCH-129', 'KCH-135', 'LDH007', 'LDH016', 'LDH017', 'LDH043', 'LDH047', 'LDH054', 'LIS15', 'RFL055']
